In [ ]:
#some introduction of what is it about ...

### The notebook helps you to run a simulation of the cloud compute costs for using an autoscaling Jupyter hub deployment.

In [ ]:
# What allows us to make changes to the z2jh_cost_simulator package
# and workaround a caching of the module, so our changes can be seen.
%load_ext autoreload
%autoreload 2

In [ ]:
#Import the simulator.py module from the z2jh_cost_simulator package
from z2jh_cost_simulator import simulator
from z2jh_cost_simulator import generate_user_activity

## Set the configurations for the simulation.
1. Info about a single node pool where users reside
    - CPU / Memory, for example 4 CPU cores / 26 GB memory node
    - Autoscaling limits, for example 0-5 nodes
    - Cost, for example 120 USD / month and node
    - Cluster autoscaler details, how long of node inactivity is required(10 minutes) 
1. Info about the resource requests (guaranteed resources) for the users
1. How much time before a user pod is culled by inactivity 
1. The  max lifetime for the user pod before it can be culled down.


In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, VBox

style_for_widget  = {'description_width': '200px'}
layout_for_widget = {'width': '400px'}
box_layout        = Layout(
                            border='solid',
                            width='500px'
                          )
heading = widgets.HTML(value="<b>Select the simulation configuration.</b>")

max_min_nodes = widgets.IntRangeSlider(min=0, max=10, step=1, description='Number of nodes:', value=(1,3), style=style_for_widget, layout=layout_for_widget,)
node_cpu        = widgets.IntSlider(min=1, max=16, description="Node CPU", value=4, style=style_for_widget, layout=layout_for_widget,)
user_pod_cpu    = widgets.FloatSlider(min=0, max=5, step =.05, value=.04, description="User pod CPU", style=style_for_widget, layout=layout_for_widget,)
node_memory     = widgets.FloatSlider(min=0, max=128.0, step =1, value=5.0, description="Node Memory(in GB)", style=style_for_widget, layout=layout_for_widget,)
user_pod_memory = widgets.FloatSlider(min=0, max=5000.0, step=64, value=1408, description="User pod Memory(in MB)", style=style_for_widget, layout=layout_for_widget,)
cost_per_month  = widgets.FloatSlider(min=0, max=150.0, step=5.5, value=12.8, description="Cost per month(in USD)", style=style_for_widget, layout=layout_for_widget,)
node_stop_time  = widgets.IntSlider(min=1, max=100, step=20, value=10, description="Node stop time", style=style_for_widget, layout=layout_for_widget,)
pod_culling_max_inactivity_time  = widgets.IntSlider(min=1, max=100, step=5, value=10, description="Pod culling for inactivity", style=style_for_widget, layout=layout_for_widget,)
pod_culling_max_lifetime         = widgets.IntSlider(min=0, max=300, step=20, value=40, description="Pod culling for max lifetime", style=style_for_widget, layout=layout_for_widget,)

simulation_configurations = [
    heading,
    max_min_nodes,
    node_cpu,
    user_pod_cpu,
    node_memory,
    user_pod_memory,
    cost_per_month,
    pod_culling_max_inactivity_time,
    pod_culling_max_lifetime,
    node_stop_time,
]

VBox(simulation_configurations, layout=box_layout)

In [ ]:
#The configurations which will be passed to the Simulation object.
simulation_configurations = {
    'min_nodes'   : max_min_nodes.lower,
    'max_nodes'   :max_min_nodes.upper,
    'node_cpu'    : node_cpu.value,
    'node_memory' : node_memory.value,
    'user_pod_cpu':user_pod_cpu.value,
    'user_pod_memory':user_pod_memory.value,
    'cost_per_month': cost_per_month.value,
    'pod_inactivity_time':pod_culling_max_inactivity_time.value,
    'pod_max_lifetime':pod_culling_max_lifetime.value,
    'node_stop_time':node_stop_time.value,
}


In [ ]:
# Hardcoded test values to be removed later, or replaced with ...
# eh... you help me describe this :)
# A full weeks worth of input to the user activity generator is provided here.

#List of number of users per hour for one weekday.
no_users_on_weekday = [2,3,4,2,3,4,5,5,4,5,5,5,5,3,3,3,3,3,4,4,4,4,4,4]
#List of number of users per hour for one week-end day.
no_users_on_weekend = [2,2,2,2,2,3,3,3,3,3,4,4,4,3,3,3,3,3,5,5,6,6,6,6]  

hour_wise_users_full_week = []
hour_wise_users_full_week.extend(no_users_on_weekday * 5)  
hour_wise_users_full_week.extend(no_users_on_weekend * 2) 

#Generate a user activity list with the hour_wise_users by calling the generate_user_activity function.
user_activity = generate_user_activity.generate_user_activity(hour_wise_users_full_week)


In [ ]:
#Create an object of the Simulation, pass the configurations and the user activity to the simulation object.
sim = simulator.Simulation(simulation_configurations, user_activity)

#The run_simulation logic runs the simulation for one week. 
# TODO: make the simulator run to the end if stop isn't assigned specifically
sim.run(stop=len(hour_wise_users_full_week)*60)

# The cluster utilization data stores the node wise utilization information for each minute.
cluster_utilization_data = sim.create_utilization_data()

In [ ]:
cluster_utilization_data

### The line chart helps you to visualize the utilization of the cluster nodes and the cluster auto scaling for one week.

In [ ]:
#imports required for the visualization of the simulation 
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode , plot, iplot
from ipywidgets import Layout, VBox

widget_style = {'description_width': 'initial'}
selected_day = widgets.IntSlider(min=1, max=7, description="Day of the week", style=widget_style)
list_nodes = list(node for node in cluster_utilization_data.columns if node.find('percent') != -1)
data = []
for node in list_nodes:
    data.append(
        go.Scatter(
                x = cluster_utilization_data['time'],
                y = cluster_utilization_data[node],
                mode = 'lines',
                name = 'Node' + str(list_nodes.index(node)+1)
              ))

g = go.FigureWidget(data=data,
                        layout=go.Layout(
                        title=dict(
                            text='utilization data'
                        ),
                       xaxis=dict(
                            title      = 'Time in (hours)',
                            tickmode   = 'array',
                            tickangle  = 45,
                            tickvals   = list(range(0,10080,120)),
                            ticktext   = [str(i) + " hours" for i in range(0,24,2)]
                               ),
                      yaxis=dict(
                        title='utilization(%)'
                        ) 
                    ))



def response(change):
    for node in list_nodes:
        x1 = cluster_utilization_data[node][(selected_day.value-1)*1440:selected_day.value*1440]
        g.data[list_nodes.index(node)].y = x1
        
selected_day.observe(response)
container2 = widgets.HBox(
                 [
                    selected_day
                 ]
               )
widgets.VBox(
      [
        container2,
        g
      ]
    )


